# Usages of various PyTorch apis

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import display_markdown

def print_table(header, rows):
    md = ['|' + ' | '.join(header) + '|']
    for i, row in enumerate(rows):
        if i == 0:
            n_cols = len(row)
            md.append('|--'*n_cols + '|')
        md.append('| {} |'.format(' | '.join([str(cell) for cell in row])))
    display_markdown('\n'.join(md), raw=True)

def print_md(s):
    display_markdown(s, raw=True)

## Embeddings
[API Docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [2]:
# This example illustrates the usage of nn.Embedding
# Embedding is a lookup table -- to lookup a vector stored against a key (typically an integer index)
# You supply a bunch of keys, you get back corresponding bunch of vectors

# This example tries to lookup vectors corresponding to words in a sentence
# Setup: here are 100 english words. Index of 'how' is 0, index of 'to' is 1, ... index of 'two' is 99
v100 = ['how', 'to', 'her', 'at', 'up', 'see', 'in', 'thing', 'even', 'because', 'or', 
             'what', 'man', 'this', 'for', 'with', 'time', 'now', 'give', 'very', 'take', 'other', 
             'there', 'would', 'first', 'about', 'people', 'think', 'find', 'so', 'say', 'as', 
             'many', 'will', 'just', 'he', 'I', 'well', 'our', 'tell', 'out', 'have', 'can', 'its', 
             'make', 'get', 'if', 'than', 'use', 'that', 'new', 'also', 'from', 'by', 'his', 'year', 
             'do', 'some', 'the', 'no', 'a', 'those', 'she', 'come', 'one', 'their', 'more', 
             'these', 'all', 'go', 'and', 'could', 'him', 'into', 'only', 'who', 'of', 'it', 'your', 
             'not', 'you', 'here', 'when', 'on', 'which', 'then', 'know', 'them', 'my', 'me', 'we', 
             'want', 'they', 'like', 'look', 'day', 'way', 'but', 'be', 'two']

# Naturally this vocabulary is insufficient even for most common sentences. Map all out of vocabulary words to <unk>
# add <unk> to vocabulary at index 100. <unk> is the replacement for out of vocabulary words
v100.append('<unk>')

# create a dictionary to lookup word's index
w2i = {word: i for i, word in enumerate(v100)}

# define few sentences of fixed size. we wish to lookup embeddings for words in these sentences
# here are 5 sentences
sents_lang = [
    'this is my book',
    'those are your books',
    'what is your name',
    'i will be back',
    'go out and about'
]

# tranform to indexed representation of sentences
sents_indices = [[w2i.get(word, w2i['<unk>']) for word in sent.split()] for sent in sents_lang]

# print representations
print_md('#### Sentences represented as array of indices')
print_table(['Sentence', 'Word Indices'], zip(sents_lang, sents_indices))
# for s1, s2 in zip(sents_lang, sents_indices):
#     print_md('*{}* => {}'.format(s1, s2))

# convert sents_word_indices to a tensor
sents_tensor = torch.tensor(sents_indices)
print_md('#### Sentences represented as a tensor')
print(sents_tensor)
print('Shape = {}'.format(sents_tensor.shape))

#### Sentences represented as array of indices

|Sentence | Word Indices|
|--|--|
| this is my book | [13, 100, 88, 100] |
| those are your books | [61, 100, 78, 100] |
| what is your name | [11, 100, 78, 100] |
| i will be back | [100, 33, 98, 100] |
| go out and about | [69, 40, 70, 25] |

#### Sentences represented as a tensor

tensor([[ 13, 100,  88, 100],
        [ 61, 100,  78, 100],
        [ 11, 100,  78, 100],
        [100,  33,  98, 100],
        [ 69,  40,  70,  25]])
Shape = torch.Size([5, 4])


In [3]:
# create an embedding to lookup 10-dimensional vectors for each word
embedding_1 = nn.Embedding(num_embeddings=len(v100), embedding_dim=10)

In [4]:
# underneath embeddings are weights, initialized with random values ~ N(0, 1)
print_md("#### Embedding weights")
print("Shape of weights = {}".format(embedding_1.weight.shape))

# what's the embedding for the word 'who'
embed = embedding_1(torch.tensor([ w2i['who']]))
print_md("#### Embedding of 'who'")
print("{}\nShape = {}".format(embed, embed.shape))

# what's the embedding of 2nd sentence
embed = embedding_1(sents_tensor[1])
print_md("#### Embedding of the sentence: *{}*".format(sents_lang[1]))
print("{}\nShape = {}".format(embed, embed.shape))

# embeddings for all sentences
embed = embedding_1(sents_tensor)
print_md("#### Embeddings of all 5 sentences")
print("{}\nShape = {}".format(embed, embed.shape))

#### Embedding weights

Shape of weights = torch.Size([101, 10])


#### Embedding of 'who'

tensor([[-0.2825, -1.0863, -1.0729, -0.3079, -2.3465, -0.4001,  0.0555,  1.8038,
         -0.4832, -2.5920]], grad_fn=<EmbeddingBackward>)
Shape = torch.Size([1, 10])


#### Embedding of the sentence: *those are your books*

tensor([[ 0.7780,  0.9255,  0.4747,  0.2262,  1.1915,  0.3400, -0.2099, -0.2856,
         -0.2899,  0.6223],
        [ 2.3802, -0.6466, -0.5523,  1.0575,  0.7071,  0.0096, -0.5120,  0.3442,
          0.0671,  2.0389],
        [ 0.6997,  0.7869, -1.2694, -0.2669,  0.7621, -1.1303,  0.1839, -0.8978,
         -1.0707,  0.3894],
        [ 2.3802, -0.6466, -0.5523,  1.0575,  0.7071,  0.0096, -0.5120,  0.3442,
          0.0671,  2.0389]], grad_fn=<EmbeddingBackward>)
Shape = torch.Size([4, 10])


#### Embeddings of all 5 sentences

tensor([[[ 8.7368e-01,  9.1121e-01, -1.4728e+00, -1.1883e-01,  1.3988e-01,
           4.7449e-01, -2.4134e-01,  2.0292e-01,  8.0251e-03,  1.5162e-01],
         [ 2.3802e+00, -6.4662e-01, -5.5234e-01,  1.0575e+00,  7.0708e-01,
           9.6448e-03, -5.1204e-01,  3.4416e-01,  6.7109e-02,  2.0389e+00],
         [ 1.3637e+00,  9.8815e-01,  3.6328e-01, -1.7120e-01, -1.4109e+00,
          -2.0556e+00,  6.2491e-01,  1.6375e+00, -3.4429e-01,  1.3861e+00],
         [ 2.3802e+00, -6.4662e-01, -5.5234e-01,  1.0575e+00,  7.0708e-01,
           9.6448e-03, -5.1204e-01,  3.4416e-01,  6.7109e-02,  2.0389e+00]],

        [[ 7.7800e-01,  9.2552e-01,  4.7475e-01,  2.2625e-01,  1.1915e+00,
           3.4001e-01, -2.0990e-01, -2.8558e-01, -2.8991e-01,  6.2229e-01],
         [ 2.3802e+00, -6.4662e-01, -5.5234e-01,  1.0575e+00,  7.0708e-01,
           9.6448e-03, -5.1204e-01,  3.4416e-01,  6.7109e-02,  2.0389e+00],
         [ 6.9966e-01,  7.8688e-01, -1.2694e+00, -2.6687e-01,  7.6214e-01,
          -1.1303

In [5]:
# A little sophisticated embedding layer
# Suppose the embeddings we want to lookup are to be linear transformed. This can be done by implementing a custom module wrapping nn.Embedding
# This module applies a linear transformation on word embeddings of size 'in_embedding_dim', the output is embeddings of size 'out_embedding_dim'
class CustomEmbedding1(nn.Module):
    def __init__(self, num_embeddings, in_embedding_dim, out_embedding_dim):
        super(CustomEmbedding1, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, in_embedding_dim)
        self.linear = nn.Linear(in_features=in_embedding_dim, out_features=out_embedding_dim)
    
    def forward(self, input):
        return F.relu_(self.linear(self.embedding(input)))

cust_embedding_1 = CustomEmbedding1(num_embeddings=len(v100), in_embedding_dim=10, out_embedding_dim=5)

In [6]:
# what's the embedding for the word 'who'
cust_embed = cust_embedding_1(torch.tensor([ w2i['who']]))
print_md("#### Embedding of 'who'")
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

# what's the embedding of 2nd sentence
cust_embed = cust_embedding_1(sents_tensor[1])
print_md("#### Embedding of the sentence: *{}*".format(sents_lang[1]))
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

# embeddings for all sentences
cust_embed = cust_embedding_1(sents_tensor)
print_md("#### Embeddings of all 5 sentences")
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

#### Embedding of 'who'

tensor([[0.7801, 0.4595, 0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward1>)
Shape = torch.Size([1, 5])


#### Embedding of the sentence: *those are your books*

tensor([[0.0000, 1.8667, 0.7490, 1.1981, 0.4445],
        [0.4229, 0.2298, 0.0000, 0.0000, 0.5433],
        [0.5156, 0.7353, 0.6418, 0.6842, 0.6846],
        [0.4229, 0.2298, 0.0000, 0.0000, 0.5433]], grad_fn=<ReluBackward1>)
Shape = torch.Size([4, 5])


#### Embeddings of all 5 sentences

tensor([[[0.0000, 0.0157, 0.0769, 0.0000, 0.0000],
         [0.4229, 0.2298, 0.0000, 0.0000, 0.5433],
         [0.0000, 0.9904, 0.9481, 0.5331, 0.0000],
         [0.4229, 0.2298, 0.0000, 0.0000, 0.5433]],

        [[0.0000, 1.8667, 0.7490, 1.1981, 0.4445],
         [0.4229, 0.2298, 0.0000, 0.0000, 0.5433],
         [0.5156, 0.7353, 0.6418, 0.6842, 0.6846],
         [0.4229, 0.2298, 0.0000, 0.0000, 0.5433]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4229, 0.2298, 0.0000, 0.0000, 0.5433],
         [0.5156, 0.7353, 0.6418, 0.6842, 0.6846],
         [0.4229, 0.2298, 0.0000, 0.0000, 0.5433]],

        [[0.4229, 0.2298, 0.0000, 0.0000, 0.5433],
         [0.2141, 0.0000, 1.1520, 0.9001, 0.0000],
         [0.0000, 0.2403, 0.0000, 0.0072, 0.0000],
         [0.4229, 0.2298, 0.0000, 0.0000, 0.5433]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.6113, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.000

## Conv1d
[API Docs](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

In [ ]:
# This example illustrates usage of nn.conv1d

# Imagine input is a sentence of words
# We lookup embeddings for each word. If sentence length is 'l' and embeddings' size is 'n', then each sentence is a (l x n) matrix
# We want to apply a 1d convolution such that there are 'm' output channels, hence there will be 'm' kernels/filters
# if 'k' is the kernel size, then each kernel is a (k x n) matrix

# we will use the setup from above, sentenses are encoded 'sents_tensor' -- a tensor of word indices
# we now use embedding layer from above to resolve word embeddings for each word
sents_embedded = embedding_1(sents_tensor)
# shape of sents_embedded = (s x l x n); where s = batch_size (no.of sentences), l = length of sentence, n = embedding size

batch_size, sent_len, embed_size = sents_embedded.shape

out_channels = 4
kernel_size = 2

c1d = nn.Conv1d(in_channels=embed_size, out_channels=4, kernel_size=kernel_size)

# let's apply convolution on 1 sentence, which is a (sent_len x embed_size) matrix
print(c1d(sents_embedded[1].transpose(0, 1).unsqueeze(0)))
print(c1d(sents_embedded.transpose(1, 2)))

In [ ]:
print(sents_embedded.shape)
print(sents_embedded.transpose(1, 2).shape)
print(sents_embedded.transpose(1, 2)[0].shape)
print(sents_embedded.transpose(1, 2)[0].unsqueeze(0).shape)